### Zadanie 2: Pobieranie, przetwarzanie i zapisywanie danych kursów walutowych z API NBP

Cel zadania 
Celem zadania jest: 
1. Pobranie kursów walutowych z API NBP za ostatni dzień. 
2. Przekształcenie danych do formy tabelarycznej przy użyciu biblioteki pandas w Pythonie. 
3. Zapisanie danych do pliku CSV. 
4. Przygotowanie danych do dalszego wykorzystania w zadaniu 3. 

Biblioteki

In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

1. Wyznaczenie daty i pobranie danych

Wykorzystano pętlę w celu dynamicznego znalezienia ostatniego dnia, dla którego NBP opublikował tabelę kursów (z pominięciem dni wolnych od pracy). Po znalezieniu odpowiedniej daty, pobrano dane z API w formacie JSON.

In [5]:
# Punkt startowy (dzisiaj)
szukana_data = datetime.now()
znaleziono_dane = False
dane_json = []

print("Rozpoczynam poszukiwanie danych...")

# Pętla szukająca danych (maksymalnie 10 dni wstecz)
for i in range(1, 11):
    # Cofamy się o 'i' dni
    data_robocza = szukana_data - timedelta(days=i)
    data_str = data_robocza.strftime('%Y-%m-%d')
    
    # Budowa adresu URL (dynamicznie wstawiamy datę - wg wymagań z zadania start_date, end_date)
    url = f"http://api.nbp.pl/api/exchangerates/tables/A/{data_str}/{data_str}/?format=json"
    
    # Sprawdzenie API
    response = requests.get(url)
    
    if response.status_code == 200:
        print(f"--> Znaleziono tabelę A dla daty: {data_str}")
        dane_json = response.json()
        znaleziono_dane = True
        break # Przerywamy pętlę (znaleziono dane)
    else:
        print(f"   Brak danych dla {data_str} (np. weekend/święto). Szukam dalej...")

# Jeśli pętla nic nie znalazła (np. awaria internetu)
if not znaleziono_dane:
    print("Nie udało się pobrać żadnych danych z ostatnich 10 dni.")

Rozpoczynam poszukiwanie danych...
   Brak danych dla 2025-12-27 (np. weekend/święto). Szukam dalej...
   Brak danych dla 2025-12-26 (np. weekend/święto). Szukam dalej...
   Brak danych dla 2025-12-25 (np. weekend/święto). Szukam dalej...
   Brak danych dla 2025-12-24 (np. weekend/święto). Szukam dalej...
--> Znaleziono tabelę A dla daty: 2025-12-23


2. Przetwarzanie danych

Przekształcono pobrany obiekt JSON na strukturę tabelaryczną (DataFrame) przy użyciu funkcji json_normalize. Wyselekcjonowano wymagane kolumny: Kod waluty, Kurs średni oraz Datę. Dostosowano nazwy kolumn do języka polskiego.

In [6]:
if znaleziono_dane:
    # Konwersja zagnieżdżonych danych JSON (klucz 'rates') do postaci tabelarycznej
    tabela_walut = pd.json_normalize(dane_json[0]['rates'])
    
    # Dołączenie kolumny z datą publikacji
    tabela_walut['effectiveDate'] = dane_json[0]['effectiveDate']
    
    # Selekcja wymaganych kolumn
    tabela_gotowa = tabela_walut[['code', 'mid', 'effectiveDate']].copy()
    
    # Zmiana nazw kolumn na polskie odpowiedniki
    tabela_gotowa.columns = ['Kod_Waluty', 'Kurs_Sredni', 'Data']
    
    # Wyświetlenie podglądu przetworzonych danych
    print("Przetworzono dane. Podgląd tabeli:")
    display(tabela_gotowa.head())
else:
    print("Nie przetworzono danych z powodu braku wyników.")

Przetworzono dane. Podgląd tabeli:


,Kod_Waluty,Kurs_Sredni,Data
0,THB,0.1152,2025-12-23
1,USD,3.5848,2025-12-23
2,AUD,2.3992,2025-12-23
3,HKD,0.4609,2025-12-23
4,CAD,2.6162,2025-12-23


3. Zapisanie danych do pliku CSV

In [7]:
if znaleziono_dane:
    nazwa_pliku = 'kursy_walut.csv'
    
    # Zapisanie tabeli do pliku CSV
    tabela_gotowa.to_csv(nazwa_pliku, index=False)
    
    print(f"Zapisano dane do pliku: {nazwa_pliku}")
else:
    print("Pominięto zapis pliku.")

Zapisano dane do pliku: kursy_walut.csv
